Densenet169

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import glob
import cv2
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from skimage.io import imread, imsave
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.applications import vgg16
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

import tensorflow as tf
import tensorflow.keras.backend as K
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import shutil
import glob

In [ ]:
!wget https://www.dropbox.com/s/9019cpkapt5zm4w/Pic_OralCancer.rar?dl=0

--2023-06-24 16:51:22--  https://www.dropbox.com/s/9019cpkapt5zm4w/Pic_OralCancer.rar?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/9019cpkapt5zm4w/Pic_OralCancer.rar [following]
--2023-06-24 16:51:22--  https://www.dropbox.com/s/raw/9019cpkapt5zm4w/Pic_OralCancer.rar
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce8d0cde96705ac65fd5f8950f3.dl.dropboxusercontent.com/cd/0/inline/B-l6en3HHaZK97o-FGTSH6gzTMdUbsPu7U2QnZTyPTWD22mDtrKJiC06IOuq971JZQHhhs5G6iPJEoGmWZk-VHILcIc7UsRUQ_1HI1cnzVnVjXE-0ioaiVmTvM78yGQ6nCInhg7cc6FTNCFQwSiqfivKEEdZJLxwZ3xGkzFGOQFtZA/file# [following]
--2023-06-24 16:51:22--  https://uce8d0cde96705ac65fd5f8950f3.dl.dropboxusercontent.com/cd/0/inline/B-l6en3HHaZK97o-FGTSH6gzTMdUbsPu7U2QnZTyPTWD22mDtrKJiC06I

In [ ]:
!pip install patool
import patoolib
patoolib.extract_archive("/content/Pic_OralCancer.rar?dl=0")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 6.4 MB/s eta 0:00:00
patool: Extracting /content/Pic_OralCancer.rar?dl=0 ...
patool: running /usr/bin/unrar x -- /content/Pic_OralCancer.rar?dl=0
patool:     with cwd='./Unpack_6mz1_1ni'
patool: ... /content/Pic_OralCancer.rar?dl=0 extracted to `Pic_OralCancer'.


'Pic_OralCancer'

In [ ]:
# Count number of images in respective classes
ROOT_DIR = "/content/Pic_OralCancer"
number_of_images = {}
for dir in os.listdir(ROOT_DIR):
  number_of_images[dir] = len(os.listdir(os.path.join(ROOT_DIR, dir)))

image_count=number_of_images.items()

In [ ]:
def datafolder(p,split):
  if not os.path.exists("./"+p):
    os.mkdir("./"+p)

  for dir in os.listdir(ROOT_DIR):
    os.makedirs("./"+p+"/"+dir)
    for img in np.random.choice(a = os.listdir(os.path.join(ROOT_DIR, dir)) ,
                                size =(math.floor(split*number_of_images[dir])-5),
                                replace=False ):
      O = os.path.join(ROOT_DIR,dir,img)  #path
      D = os.path.join("./"+p,dir)
      shutil.copy(O,D)
      os.remove(O)

  else :
    print(f"{p}Folder exists")


In [ ]:
datafolder("train",0.7)
datafolder("val",0.15)
datafolder("test",0.15)

trainFolder exists
valFolder exists
testFolder exists


In [ ]:
train_dir='/content/train'
validation_dir='/content/val'
train_cancer=os.path.join(train_dir,'cancer')
train_noncancer=os.path.join(train_dir,'non-cancer')
val_cancer=os.path.join(validation_dir,'cancer')
val_noncancer=os.path.join(validation_dir,'non-cancer')

In [ ]:
train_c=os.listdir(train_cancer)
train_nc=os.listdir(train_noncancer)
val_c=os.listdir(val_cancer)
val_nc=os.listdir(val_noncancer)

print(len(train_c))
print(len(train_nc))
print(len(val_c))
print(len(val_nc))

55
25
8
1


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale = 1./255.,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

test_datagen = ImageDataGenerator(
    rescale = 1./255.
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    batch_size=10,
    class_mode='binary',
    target_size=(150,150)
)

validation_generator =  test_datagen.flow_from_directory(
    validation_dir,
    batch_size=10,
    class_mode='binary',
    target_size=(150,150)
)

Found 80 images belonging to 2 classes.
Found 9 images belonging to 2 classes.


In [ ]:
pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 33.8 MB/s eta 0:00:00


In [ ]:
import tensorflow_addons as tfa


def Ranger(sync_period=6,
           slow_step_size=0.5,
           learning_rate=0.001,
           beta_1=0.9,
           beta_2=0.999,
           epsilon=1e-7,
           weight_decay=0.,
           amsgrad=False,
           sma_threshold=5.0,
           total_steps=0,
           warmup_proportion=0.1,
           min_lr=0.,
           name="Ranger"):
    # create RAdam optimizer
    inner = tfa.optimizers.RectifiedAdam(learning_rate, beta_1, beta_2, epsilon, weight_decay, amsgrad, sma_threshold, total_steps, warmup_proportion, min_lr, name)
    # feed RAdam optimizer into lookahead operation
    optim = tfa.optimizers.Lookahead(inner, sync_period, slow_step_size, name)
    return optim

In [ ]:
from tensorflow.keras.applications import DenseNet169
def build_model():
  base_model = DenseNet169(input_shape=(224,224,3), weights='imagenet', include_top=False)
  output = layers.GlobalAveragePooling2D()(base_model.output)
  output = layers.Dense(2, activation='softmax')(output)
  model = Model(base_model.input, output)

  for layer in base_model.layers[:-4]:
      layer.trainable=False

  # optimizer = tf.keras.optimizers.RMSprop(0.001)
  optimizer=Ranger()

  model.compile(loss='sparse_categorical_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy'])

  model.summary()

  return model

In [ ]:
model=build_model()

51877672/51877672 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                             